# Neural Style Transfer Using PyTorch
## Introduction
Neural style transfer is an artificial system based on the Deep Neural Network to generate artistic images. This approach uses two random images, the content and the style image. It extracts the structural features from the content image, whereas the style features from the style image.


# Content and Style Representation
### Content Representation
The convolutional neural network develops representations of the image along the processing hierarchy. As we move deeper into the network, the representations will care more about the structural features or the actual content than the detailed pixel data. To obtain these representations, we can reconstruct the images using the feature maps of that layer. Reconstruction from the lower layer will reproduce the exact image. In contrast, the higher layer's reconstruction will capture the high-level content and hence we refer to the feature responses from the higher layer as the content representation.

### Style Representation
To extract the representation of the style content, we build a feature space on the top of the filter responses in each network layer. It consists of the correlations between the different filter responses over the spatial extent of the feature maps. The filter correlation of different layers captures the texture information of the input image. This creates images that match a given image's style on an increasing scale while discarding information of the global arrangement. This multi-scale representation is called style representation.

# Implementation
### Import Libraries

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torchvision.utils import save_image

### Load the model

In [3]:
#Loadung the model vgg19 that will serve as the base model
model=models.vgg19(pretrained=True).features
#Assigning the GPU to the variable device
device=torch.device( "cuda" if (torch.cuda.is_available()) else 'cpu')

### Image Preprocessing
To make an image compatible with the model, the image has to be preprocessed. Using the torch.transforms() we will perform some of the basic preprocessing that will include the following steps:
* Resize: resize all the image to 512 x 512
* ToTensor: to convert the image into a tensor

You can also Normalize the tensor with mean (0.485, 0.456, 0.406) and standard deviation of (0.229, 0.224, 0.225) for the pretrained vgg19 model. But don’t forget to convert it back to its original scale. So, define a function that loads the image using the PIL library and preprocess it. Add an extra dimension at 0th index, using unsqueeze() for batch size and then load it to the device and return it.

Now, use the image_loader function to load the style and the content image from the local disk. It’s a general practice to use the content image clone as the input base image or the generated image. Since the gradient descent will alter the generated image's pixel values, we will pass the parameter true for require_grads().

In [5]:
#defing a function that will load the image and perform the required preprocessing and put it on the GPU
def image_loader(path):
    image=Image.open(path)
    #defining the image transformation steps to be performed before feeding them to the model
    loader=transforms.Compose([transforms.Resize((512,512)), transforms.ToTensor()])
    #The preprocessing steps involves resizing the image and then converting it to a tensor
    image=loader(image).unsqueeze(0)
    return image.to(device,torch.float)

#Loading the original and the style image
original_image=image_loader('NikolaTesla.jpg')
style_image=image_loader('style.jpeg')

#Creating the generated image from the original image
generated_image=original_image.clone().requires_grad_(True)

### Get Feature Representations
Let’s define a class that will provide the feature representations of the intermediate layers. Intermediate layers are used because these layers serve as a complex feature extractor. Hence these can describe the style and content of the input image. In this class, we will initialize a model by eliminating the unused layers( layers beyond conv5_1) of the vgg19 model and extract the activations or the feature representations of the ‘conv1_1’, ‘conv2_1’, ‘conv3_1’, ‘conv4_1’ and ‘conv5_1’ layers (index values [0, 5, 10, 19, 28]). Store these activations of 5 convolutional layers in an array and return the array.

In [6]:
#Defining a class that for the model
class VGG(nn.Module):
    def __init__(self):
        super(VGG,self).__init__()
        self.req_features= ['0','5','10','19','28'] 
        #Since we need only the 5 layers in the model so we will be dropping all the rest layers from the features of the model
        self.model=models.vgg19(pretrained=True).features[:29] #model will contain the first 29 layers
    
   
    #x holds the input tensor(image) that will be feeded to each layer
    def forward(self,x):
        #initialize an array that wil hold the activations from the chosen layers
        features=[]
        #Iterate over all the layers of the mode
        for layer_num,layer in enumerate(self.model):
            #activation of the layer will stored in x
            x=layer(x)
            #appending the activation of the selected layers and return the feature array
            if (str(layer_num) in self.req_features):
                features.append(x)
                
        return features

### Define the Loss
The net loss for style transfer is defined as :
![title](src/totalLoss.png)
In the above equation, Lₜₒₜₐₗ is the total loss, L 𝒸ₒₙₜₑₙₜ is the content loss of all the intermediate layers and Lₛₜᵧₗₑ is the style loss of all the intermediate layers. Here, α and β are the weighting coefficients of the content and the style loss, respectively. p, a and x are the content image, style image and the generated image or the base input image. We perform gradient descent on the loss function and instead of the model parameters we update the pixel values of the input image x to minimize the loss. This will make the input image similar to the content and the style image. We can emphasize either the style or the content loss by altering the values of α and β. A strong emphasis on style will result in images that match the artwork's appearance, effectively giving a texturized version of it, but hardly show any of the photograph’s content. When placing a strong emphasis on content, one can identify the photograph, but the painting style is not as well-matched.

### Content Loss
The content image and the input base image are passed to our model and the intermediate layers' outputs (listed above as ‘conv1_1’, ‘conv2_1’, ‘conv3_1’, ‘conv4_1’ and ‘conv5_1’) are extracted using the above-defined class. Then we calculate the euclidean distance between the intermediate representation of the content image and the input base image. Hence the content loss for a layer l is defined by:
![title](src/contentLoss.png)
In the above equation, we calculate the squared error between the feature representation of the content image (p) and the input base image (x) of layer (l). Here, nˡₕ, nˡ𝓌, nˡ𝒸 are the height, width and channels of layer l. To calculate the content loss, the intermediate representation of dimensions nˡₕ x nˡ𝓌 x nˡ𝒸 is unrolled into vectors of dimensions nˡ𝒸 x nˡₕ*nˡ𝓌. Unrolling the feature representation is not a compulsory step, but it’s a good practice. The following diagram will help us to visualize this transformation.

![title](src/contentLoss_diagram.png)

Fˡᵢⱼ and Pˡᵢⱼ are the nˡ𝒸 x nˡₕ*nˡ𝓌 dimension vectors representing the intermediate representation of the input base image and the content image.

In [7]:
def calc_content_loss(gen_feat,orig_feat):
    #calculating the content loss of each layer by calculating the MSE between the content and generated features and adding it to content loss
    content_l=torch.mean((gen_feat-orig_feat)**2)
    return content_l

### Style Loss
We build a feature space over each layer of the network representing the correlation between the different filter responses. The Gram matrix calculates these feature correlations.
The gram matrix represents the correlation between each filter in an intermediate representation. Gram matrix is calculated by taking the dot product of the unrolled intermediate representation and its transpose. The gram matrix G dimension is nˡ𝒸 x nˡ𝒸, where nˡ𝒸 is the number of channels in the intermediate representation of layer l.
![title](src/styleLoss.png)
In the above equation, Gˡᵢⱼ is the inner product between the vectorized feature map i and j in layer l. The vectorized equation of a gram matrix is shown in the following figure, where G is the gram matrix of intermediate representation A.
![title](src/styleLoss_diagram.png)
![title](src/style_abstract_equ.png)
The style loss of layer l is the squared error between the gram matrices of the intermediate representation of the style image and the input base image.
![title](src/styleLossPart2.png)
Where Eₗ is the style loss for layer l, Nₗ and Mₗ are the numbers of channels and height times width in the feature representation of layer l respectively. Gˡᵢⱼ and Aˡᵢⱼ are the gram matrices of the intermediate representation of the style image (a) and the input base image (x) respectively.
The total style loss is :
![title](src/styleLossPart3.png)
Where wˡ is the weight factor of the contribution of each layer to the total style loss.

In [8]:

def calc_style_loss(gen,style):
    #Calculating the gram matrix for the style and the generated image
    batch_size,channel,height,width=gen.shape

    G=torch.mm(gen.view(channel,height*width),gen.view(channel,height*width).t())
    A=torch.mm(style.view(channel,height*width),style.view(channel,height*width).t())
        
    #Calcultating the style loss of each layer by calculating the MSE between the gram matrix of the style image and the generated image and adding it to style loss
    style_l=torch.mean((G-A)**2)
    return style_l

Here we are not multiplying the style loss with the constant as it makes the loss very small. Sum the content and the style loss for all the intermediate layers and calculate the total loss.

In [9]:

def calculate_loss(gen_features, orig_feautes, style_featues):
    style_loss=content_loss=0
    for gen,cont,style in zip(gen_features,orig_feautes,style_featues):
        #extracting the dimensions from the generated image
        content_loss+=calc_content_loss(gen,cont)
        style_loss+=calc_style_loss(gen,style)
    
    #calculating the total loss of e th epoch
    total_loss=alpha*content_loss + beta*style_loss 
    return total_loss

## Training
Initialize the variables that we will need to train our model. So, before moving on to train, we need to set the hyperparameters.

In [10]:
#Load the model to the GPU
model=VGG().to(device).eval() 

#initialize the paramerters required for fitting the model
epoch=7000
lr=0.004
alpha=8
beta=70

#using adam optimizer and it will update the generated image not the model parameter 
optimizer=optim.Adam([generated_image],lr=lr)

Here, create an object for the class VGG. Initializing the object will call the constructor and it will return a model with the first 29 layers and load it to the device. Epoch is 1000, the learning rate is 0.01, alpha(weighting coefficient of content loss) is 1 and beta(weighting coefficient of style loss) is 0.01.
Adam is used as an optimizer. The pixel data of the generated image will be optimized to pass the generated image as the optimizer parameter.
Use a for loop to iterate over the number of epochs. Extract the feature representation of the intermediate layers of the content, style and the generated image using the model. On passing an image to the model, it will return an array of length 5. Each element corresponds to the feature representation of each intermediate layer.
Calculate the total loss by using the above-defined functions. Set the gradients to zero with optimizer.zero_grads(), backpropagate the total loss with total_loss.backward() and then update the pixel values of the generated image using optimizer.step(). We will save the generated image after every 100 epochs and print the total loss.

In [11]:
#iterating for 1000 times
for e in range (epoch):
    #extracting the features of generated, content and the original required for calculating the loss
    gen_features=model(generated_image)
    orig_feautes=model(original_image)
    style_featues=model(style_image)
    
    #iterating over the activation of each layer and calculate the loss and add it to the content and the style loss
    total_loss=calculate_loss(gen_features, orig_feautes, style_featues)
    #optimize the pixel values of the generated image and backpropagate the loss
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()
    #print the image and save it after each 100 epoch
    if(e/100):
        print(total_loss)
        
        save_image(generated_image,"gen.png")   

tensor(1.4184e+10, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.2122e+10, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.0505e+10, device='cuda:0', grad_fn=<AddBackward0>)
tensor(9.2121e+09, device='cuda:0', grad_fn=<AddBackward0>)
tensor(7.9424e+09, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.8838e+09, device='cuda:0', grad_fn=<AddBackward0>)
tensor(6.0404e+09, device='cuda:0', grad_fn=<AddBackward0>)
tensor(5.3499e+09, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.7778e+09, device='cuda:0', grad_fn=<AddBackward0>)
tensor(4.2892e+09, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.8451e+09, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.4254e+09, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.0381e+09, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.6971e+09, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.4070e+09, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.1654e+09, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.9678e+09, device='cuda:0', grad

tensor(2.4410e+08, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.4148e+08, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.3890e+08, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.3634e+08, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.3380e+08, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.3130e+08, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.2882e+08, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.2637e+08, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.2394e+08, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.2154e+08, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.1917e+08, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.1682e+08, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.1450e+08, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.1221e+08, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.0994e+08, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.0769e+08, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.0547e+08, device='cuda:0', grad

tensor(82366760., device='cuda:0', grad_fn=<AddBackward0>)
tensor(82088080., device='cuda:0', grad_fn=<AddBackward0>)
tensor(81813888., device='cuda:0', grad_fn=<AddBackward0>)
tensor(81544040., device='cuda:0', grad_fn=<AddBackward0>)
tensor(81278496., device='cuda:0', grad_fn=<AddBackward0>)
tensor(81017168., device='cuda:0', grad_fn=<AddBackward0>)
tensor(80760000., device='cuda:0', grad_fn=<AddBackward0>)
tensor(80506824., device='cuda:0', grad_fn=<AddBackward0>)
tensor(80257472., device='cuda:0', grad_fn=<AddBackward0>)
tensor(80011840., device='cuda:0', grad_fn=<AddBackward0>)
tensor(79770008., device='cuda:0', grad_fn=<AddBackward0>)
tensor(79531640., device='cuda:0', grad_fn=<AddBackward0>)
tensor(79296752., device='cuda:0', grad_fn=<AddBackward0>)
tensor(79065296., device='cuda:0', grad_fn=<AddBackward0>)
tensor(78837240., device='cuda:0', grad_fn=<AddBackward0>)
tensor(78612272., device='cuda:0', grad_fn=<AddBackward0>)
tensor(78390488., device='cuda:0', grad_fn=<AddBackward0

tensor(61720784., device='cuda:0', grad_fn=<AddBackward0>)
tensor(61624600., device='cuda:0', grad_fn=<AddBackward0>)
tensor(61528752., device='cuda:0', grad_fn=<AddBackward0>)
tensor(61433200., device='cuda:0', grad_fn=<AddBackward0>)
tensor(61338016., device='cuda:0', grad_fn=<AddBackward0>)
tensor(61243124., device='cuda:0', grad_fn=<AddBackward0>)
tensor(61148552., device='cuda:0', grad_fn=<AddBackward0>)
tensor(61054292., device='cuda:0', grad_fn=<AddBackward0>)
tensor(60960412., device='cuda:0', grad_fn=<AddBackward0>)
tensor(60866832., device='cuda:0', grad_fn=<AddBackward0>)
tensor(60773552., device='cuda:0', grad_fn=<AddBackward0>)
tensor(60680572., device='cuda:0', grad_fn=<AddBackward0>)
tensor(60587960., device='cuda:0', grad_fn=<AddBackward0>)
tensor(60495676., device='cuda:0', grad_fn=<AddBackward0>)
tensor(60403628., device='cuda:0', grad_fn=<AddBackward0>)
tensor(60311984., device='cuda:0', grad_fn=<AddBackward0>)
tensor(60220552., device='cuda:0', grad_fn=<AddBackward0

tensor(50883872., device='cuda:0', grad_fn=<AddBackward0>)
tensor(50821120., device='cuda:0', grad_fn=<AddBackward0>)
tensor(50758468., device='cuda:0', grad_fn=<AddBackward0>)
tensor(50695944., device='cuda:0', grad_fn=<AddBackward0>)
tensor(50633692., device='cuda:0', grad_fn=<AddBackward0>)
tensor(50571556., device='cuda:0', grad_fn=<AddBackward0>)
tensor(50509596., device='cuda:0', grad_fn=<AddBackward0>)
tensor(50447772., device='cuda:0', grad_fn=<AddBackward0>)
tensor(50386172., device='cuda:0', grad_fn=<AddBackward0>)
tensor(50324736., device='cuda:0', grad_fn=<AddBackward0>)
tensor(50263460., device='cuda:0', grad_fn=<AddBackward0>)
tensor(50202340., device='cuda:0', grad_fn=<AddBackward0>)
tensor(50141428., device='cuda:0', grad_fn=<AddBackward0>)
tensor(50080668., device='cuda:0', grad_fn=<AddBackward0>)
tensor(50020012., device='cuda:0', grad_fn=<AddBackward0>)
tensor(49959588., device='cuda:0', grad_fn=<AddBackward0>)
tensor(49899244., device='cuda:0', grad_fn=<AddBackward0

...

tensor(3157861.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3178578.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3190845., device='cuda:0', grad_fn=<AddBackward0>)
tensor(3190633.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3204700.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3235052.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3266322.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3288356.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3311097.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3322781.7500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3322202., device='cuda:0', grad_fn=<AddBackward0>)
tensor(3303418.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3286871., device='cuda:0', grad_fn=<AddBackward0>)
tensor(3259100.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3217983.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3171058.7500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3145242.5000,

tensor(3173435.7500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3159745.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3168251., device='cuda:0', grad_fn=<AddBackward0>)
tensor(3147516.7500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3154172.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3150435.7500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3138279.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3144062., device='cuda:0', grad_fn=<AddBackward0>)
tensor(3133460.7500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3132367.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3130467.7500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3123551.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3123433.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3119049.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3116428.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3113934.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3110117.,

tensor(3106574.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3082588.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3091115., device='cuda:0', grad_fn=<AddBackward0>)
tensor(3156693.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3198358.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3207514., device='cuda:0', grad_fn=<AddBackward0>)
tensor(3191925.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3192499.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3162779., device='cuda:0', grad_fn=<AddBackward0>)
tensor(3098648.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3036428.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3014094.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3011042., device='cuda:0', grad_fn=<AddBackward0>)
tensor(3006629.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3018078.7500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3046445.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3060343.5000, dev

tensor(2965885., device='cuda:0', grad_fn=<AddBackward0>)
tensor(2971966.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2953454.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2960868.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2955113., device='cuda:0', grad_fn=<AddBackward0>)
tensor(2940497.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2948256.7500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2952547.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2971484.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3014444.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3063873.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3160625.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3251271.7500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3320373.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3269861.7500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3126329.7500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2967206.,

tensor(3206301.7500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3209804.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3197719.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3200127., device='cuda:0', grad_fn=<AddBackward0>)
tensor(3180875., device='cuda:0', grad_fn=<AddBackward0>)
tensor(3175774., device='cuda:0', grad_fn=<AddBackward0>)
tensor(3150109.2500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3139630.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3115799.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3109133., device='cuda:0', grad_fn=<AddBackward0>)
tensor(3098810.7500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3095787.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3093516., device='cuda:0', grad_fn=<AddBackward0>)
tensor(3094721.7500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3095996.5000, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3097411.7500, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3100762.2500, device=

After training, you can find the artistic image in your current working directory